### Imports

In [46]:
import pandas as pd
import numpy as np
import sklearn
import pickle

### Loading Dataset

In [118]:
data_train = pd.read_csv('train.csv')


In [119]:
data_test=pd.read_csv('test.csv')
sub=pd.read_csv('gender_submission.csv')

In [121]:
data_train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
784,785,0,3,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.0500,NaN,S
554,555,1,3,"Ohman, Miss. Velin",female,22.0,0,0,347085,7.7750,NaN,S
639,640,0,3,"Thorneycroft, Mr. Percival",male,NaN,1,0,376564,16.1000,NaN,S
498,499,0,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0,1,2,113781,151.5500,C22 C26,S
837,838,0,3,"Sirota, Mr. Maurice",male,NaN,0,0,392092,8.0500,NaN,S
291,292,1,1,"Bishop, Mrs. Dickinson H (Helen Walton)",female,19.0,1,0,11967,91.0792,B49,C
428,429,0,3,"Flynn, Mr. James",male,NaN,0,0,364851,7.7500,NaN,Q
312,313,0,2,"Lahtinen, Mrs. William (Anna Sylfven)",female,26.0,1,1,250651,26.0000,NaN,S
604,605,1,1,"Homer, Mr. Harry (""Mr E Haven"")",male,35.0,0,0,111426,26.5500,NaN,C
727,728,1,3,"Mannion, Miss. Margareth",female,NaN,0,0,36866,7.7375,NaN,Q


In [122]:
column_names = data_train.columns
for column in column_names:
    print(column + ' - ' + str(data_train[column].isnull().sum()))

PassengerId - 0
Survived - 0
Pclass - 0
Name - 0
Sex - 0
Age - 177
SibSp - 0
Parch - 0
Ticket - 0
Fare - 0
Cabin - 687
Embarked - 2


### Pre-processing Data

In [65]:
# def pre_process(dat):
#     data=dat
#     data['Age'].fillna(data['Age'].mean(), inplace = True)
#     data['Fare'].fillna(data['Fare'].mean(), inplace = True)
#     data['Embarked'].fillna(data['Embarked'].bfill(), inplace = True)
#     data['Sex'] = data['Sex'].map({'male':0, 'female':1})
#     data['Embarked'] = data['Embarked'].map({'C':0, 'Q':1, 'S':2})
#     test= data.drop([ 'Name','Ticket', 'Cabin'], axis = 1)
#     return test

In [123]:
data_train['Age'].fillna(data_train['Age'].mean(), inplace = True)
data_train['Fare'].fillna(data_train['Fare'].mean(), inplace = True)
data_train['Embarked'].fillna(data_train['Embarked'].bfill(), inplace = True)

data_test['Age'].fillna(data_test['Age'].mean(), inplace = True)
data_test['Fare'].fillna(data_test['Fare'].mean(), inplace = True)
data_test['Embarked'].fillna(data_test['Embarked'].bfill(), inplace = True)

In [124]:
column_names = data_train.columns
for column in column_names:
    print(column + ' - ' + str(data_train[column].isnull().sum()))

PassengerId - 0
Survived - 0
Pclass - 0
Name - 0
Sex - 0
Age - 0
SibSp - 0
Parch - 0
Ticket - 0
Fare - 0
Cabin - 687
Embarked - 0


In [125]:
data_train['Sex'] = data_train['Sex'].map({'male':0, 'female':1})
data_train['Embarked'] = data_train['Embarked'].map({'C':0, 'Q':1, 'S':2})
data_test['Sex'] = data_test['Sex'].map({'male':0, 'female':1})
data_test['Embarked'] = data_test['Embarked'].map({'C':0, 'Q':1, 'S':2})

In [126]:
passenger_test=pd.DataFrame(data_test['PassengerId'])
y_train=data_train['Survived']
y_test=sub['Survived']
x_train = data_train.drop(['PassengerId','Survived', 'Name', 'Ticket', 'Cabin'], axis = 1)
x_test= data_test.drop([ 'PassengerId','Name','Ticket', 'Cabin'], axis = 1)
x_train.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
41,2,1,27.000000,1,0,21.0000,2
175,3,0,18.000000,1,1,7.8542,2
617,3,1,26.000000,1,0,16.1000,2
46,3,0,29.699118,1,0,15.5000,1
104,3,0,37.000000,2,0,7.9250,2
218,1,1,32.000000,0,0,76.2917,0
207,3,0,26.000000,0,0,18.7875,0
503,3,1,37.000000,0,0,9.5875,2
11,1,1,58.000000,0,0,26.5500,2
760,3,0,29.699118,0,0,14.5000,2


In [127]:
x_test.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
82,1,0,49.00000,0,0,26.0000,2
36,3,1,30.27259,0,0,8.0500,2
400,1,1,30.00000,0,0,164.8667,2
151,3,0,30.27259,0,0,7.8958,0
91,3,0,30.27259,0,0,7.7750,2
319,2,0,22.00000,2,0,31.5000,2
315,3,1,17.00000,0,0,7.7333,1
300,3,0,32.00000,0,0,7.7750,2
402,1,1,22.00000,0,1,59.4000,0
110,2,0,41.00000,0,0,15.0458,0


In [70]:
# def remove_pk(data):
#     new_data=data.loc[ : , data.columns != 'PassengerId']
#     return new_data

### Creating Decision tree Model

In [129]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(max_depth = 10, min_samples_split = 20)
decision_tree.fit(x_train,y_train)
y_pred = decision_tree.predict(x_test)

In [109]:
class DecisionTreeModel:
    def __init__(self,max_depth=10,min_samples_split=20):
        self.model = DecisionTreeClassifier(max_depth=max_depth,min_samples_split=min_samples_split)
    def pre(self,dat):
        data=dat.copy(deep=True)
        data['Age'].fillna(data['Age'].mean(), inplace = True)
        data['Fare'].fillna(data['Fare'].mean(), inplace = True)
        data['Embarked'].fillna(data['Embarked'].bfill(), inplace = True)
        data['Sex'] = data['Sex'].map({'male':0, 'female':1})
        data['Embarked'] = data['Embarked'].map({'C':0, 'Q':1, 'S':2})
        test= data.drop([ 'PassengerId','Name','Ticket', 'Cabin'], axis = 1)
        return test
    def fit(self,x,y):
        self.model.fit(x,y)
    def predict(self,data):
        processed=self.pre(data)
        return self.model.predict(processed)
        
    

In [111]:
# decision_tree = DecisionTreeModel()
# decision_tree.fit(remove_pk(x_train),y_train)

In [132]:
y_pred = decision_tree.predict(x_test)

In [133]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE : ',RMSE)

cm = confusion_matrix(y_test, y_pred)
print(cm)

RMSE :  0.39735970711951313
[[249  17]
 [ 49 103]]


In [26]:
#query_data=pd.concat([x_train, x_test])
#query_data

### Saving the model and pre-processed test data set

In [134]:
pickle_out = open("model.pickle","wb")
#query_data.to_csv('testing_data.csv')
#passenger_test.to_csv('passenger.csv',index=False)
pickle.dump(decision_tree, pickle_out)
pickle_out.close()

### Loading it to check validity

In [135]:
pickle_in = open("model.pickle","rb")
dt2 = pickle.load(pickle_in)
pickle_in.close()
#testing_data=pd.read_csv('testing_data.csv',index_col=[0])

In [137]:
dt2.predict(x_test)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,